## Step 1: Elasticsearch Client Setup


In [ ]:
# Bonsai credentials and URL from environment variables
BONSAI_HOST = os.getenv('BONSAI_HOST')
ACCESS_KEY = os.getenv('ACCESS_KEY')
ACCESS_SECRET = os.getenv('ACCESS_SECRET')

# Set up Elasticsearch client
es = Elasticsearch(
    [{'host': BONSAI_HOST, 'port': 443, 'use_ssl': True}],
    http_auth=(ACCESS_KEY, ACCESS_SECRET)
)

Elasticsearch is a distributed search and analytics engine often used for log aggregation, full-text search, and more. 

In this example, we're connecting to Bonsai, a managed Elasticsearch service. However this can be any elastic search setup even from your local docker environment.

Environment variables (BONSAI_HOST, ACCESS_KEY, ACCESS_SECRET) are used to securely manage sensitive data.
The Elasticsearch client is initialized, which allows interaction with Elasticsearch for sending logs from our application.

In [ ]:
app = Flask(__name__)

# Create index for logs if it doesn't exist
index_name = 'logs'
try:
    es.indices.create(index=index_name)
except NotFoundError:
    pass  # Index already exists
except Exception as e:
    print("Error creating index:", e)

#### Before logging data to Elasticsearch, we ensure that the logs index exists. Elasticsearch organizes data into indices, which are equivalent to tables in a relational database.

In [ ]:
class ElasticSearchHandler(logging.Handler):
    def emit(self, record):
        log_entry = self.format(record)
        # Prepare the log entry for Elasticsearch
        doc = {
            'timestamp': datetime.now(),
            'level': record.levelname,
            'message': log_entry,
            'service': 'my_flask_app'
        }
        # Index the log entry in Elasticsearch
        es.index(index=index_name, body=doc)

# Set up the Elasticsearch logging handler
handler = ElasticSearchHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(name)s :: %(levelname)-8s :: %(message)s')
handler.setFormatter(formatter)
app.logger.addHandler(handler)
app.logger.setLevel(logging.DEBUG)

We define a custom logging handler (ElasticSearchHandler) that sends log messages to Elasticsearch. The emit method is triggered each time a log entry is created.
The log entry is formatted and indexed into Elasticsearch under the logs index.

In [ ]:
model = LogisticRegression()

# Train the model on dummy data
X_train = np.array([[0, 0], [1, 1]])
y_train = np.array([0, 1])
model.fit(X_train, y_train)


A simple Logistic Regression model is created using scikit-learn.
It is trained on dummy data with two features corresponding to binary outputs. In a real-world application, you would train the model on larger datasets.


In [ ]:
@app.route('/predict', methods=['GET'])
def predict():
    # Get query parameters
    feature_1 = float(request.args.get('feature_1', 0))  # Default value is 0 if not provided
    feature_2 = float(request.args.get('feature_2', 0))  # Default value is 0 if not provided

    # Prepare input for the model
    input_features = np.array([[feature_1, feature_2]])

    # Make prediction
    prediction = model.predict(input_features)

    # Log the prediction
    app.logger.info(f"Prediction made: {prediction[0]} for features {input_features}")

    # Return prediction result as JSON
    return jsonify({
        'input': {
            'feature_1': feature_1,
            'feature_2': feature_2
        },
        'prediction': int(prediction[0])
    })


The /predict route accepts two query parameters (feature_1 and feature_2), which are passed to the Logistic Regression model.
The model makes a prediction based on the input features.
The prediction result is logged to Elasticsearch and returned as JSON to the client.

---

### Conclusion:
1. Ephemeral Nature: Containerized applications (such as those running in Docker) are ephemeral, meaning they can be created and destroyed at any time. If you rely on local logs within the container, you risk losing important information when the container stops or restarts.
2. Centralized and Preserved Logs: Remote logging solutions, such as Elasticsearch, allow you to centralize and preserve logs from various containers, ensuring that you have access to critical logs even after the container's lifecycle ends.
3. Lifecycle of Instances: Elastic Compute Cloud (EC2) or similar cloud infrastructure instances may be auto-scaled, stopped, or replaced. Logging locally can result in log loss as instances are replaced or terminated.

4. Persistent Storage of Logs: By sending logs to a remote logging solution like Elasticsearch, you ensure centralized access and persistent storage of logs, independent of the instance’s lifecycle.


Benefits of Remote Logging
1. Centralized Log Management: Remote logging solutions make it easier to aggregate logs from multiple containers, services, or instances. With Elasticsearch, you can easily monitor, search, and analyze logs from different sources in real-time using tools like Kibana.

2. Monitoring and Troubleshooting: Remote logs provide an easier way to track issues, bugs, or performance bottlenecks by allowing you to inspect logs from multiple containers or services in a single location. Elasticsearch enables you to create queries and dashboards for better observability and troubleshooting.

3. Compliance and Auditing: In some industries, regulations require logs to be stored for a certain period. Remote logging to Elasticsearch ensures logs are securely stored and can be retrieved if needed for audits or compliance checks.